In [1]:
print('e')

e


In [4]:
import re
import PyPDF2
import io
import google.generativeai as genaiI
from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from concurrent.futures import ThreadPoolExecutor
import os
import tempfile
import json
from langchain_community.document_loaders import PyPDFLoader
# from langchain_google_genai import GoogleGenerativeAIEmbeddings


c:\Users\UTKARSH\.conda\envs\aiagent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
pdf_path = "C:\AI-Recruitment-agent\datascienceresume32 - Google Docs.pdf"

In [6]:
loder = PyPDFLoader(pdf_path)


In [7]:
documents = loder.load()

# print(documents[0].page_content)


In [8]:
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEYY = os.getenv("GEMINI_API")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEYY)


In [6]:
result = llm.invoke('hii')
print(result.content)

Hi there! How can I help you today?



In [9]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEYY)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Maximum size of each chunk
    chunk_overlap=200,    # Overlap between chunks to maintain context
    length_function=len  # Function to measure chunk length
    
)
chunks = text_splitter.split_documents(documents)
texts = [doc.page_content for doc in chunks]  # Extract text from Document objects
vectorstore = FAISS.from_texts(texts, embeddings)



In [10]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:
{context}

Question: {input} 
""")
     



In [11]:
retriever = vectorstore.as_retriever()


In [12]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)

In [13]:
from langchain.chains import create_retrieval_chain
ra_chain = create_retrieval_chain(retriever, document_chain)


In [14]:
response = ra_chain.invoke({"input": 'What is my fathers name?'})

In [15]:
print(response['answer'])

I'm sorry, but the provided context does not contain information about your father's name.



In [19]:
print(response['context'][0].page_content)

●  Data Science Certification  -  Coursera  November 2023 
 ●  Earned Badge of Python and SQL  –  HackerRank  January 2024 
 ●  Five Star Python Coder -  HackerRank  May 2024 
 ADDITIONAL INFORMA TION 
 ●  Location :  Delhi 
 ●  Availability :  Immediate Joiner


In [16]:
skills_to_analyze = [
    "Python",
    "Java",
    "React",
    "PostgreSQL",
    "Cloud Computing (AWS/Azure/GCP)",
    "Leadership",
    "Project Management",
    "Machine Learning" # Example of a skill that might not be explicitly mentioned
]


In [21]:
# skill_scores = []
# for skill in skills_to_analyze:
#     query = f"On a scale of 0-10, how clearly does the candidate mention proficiency in {skill}? Provide a numeric rating first, followed by resoning"
#     response = retrieval_chain.invoke({"input": query})
#     match = re.search(r'(\d+)', response['answer'])
#     score = int(match.group(1) if match else 0)
#     reasoning_parts = response['answer'].split('\n', 1)
#     if len(reasoning_parts) > 1:
            
#             reasoning = reasoning_parts[1].strip()
#     else:
#             reasoning = response['answer'].strip()
#     # reasoning = response['answer'].split('\n', 1)[1].strip() if '.' in response['answer'] and len(response['answer'].split('.')) > 1 else ""
#     final_score = min(score, 10)
#     skill_scores.append(final_score)
#     print(f"  Score for {skill}: {final_score}/10")
#     print(f"  Reasoning: {reasoning}")
# # Overall score calculation
    
# # overall_score = sum(skill_scores)
# # n = len(skill_scores)
# # print(f"Overall Score: {overall_score}/{n * 10}")
# # find overll score
# print(skill_scores)





In [22]:
# final code
skill_scores = []
for skill in skills_to_analyze:
    query = f"On a scale of 0-10, how clearly does the candidate mention proficiency in {skill}? Provide a numeric rating first, followed by resoning"
    response = ra_chain.invoke({"input": query})
    match = re.search(r'(\d+)', response['answer'])
    score = int(match.group(1) if match else 0)
    reasoning_parts = response['answer']
    reasining_lines = [line.strip() for line in reasoning_parts.split('\n') if line.strip()]
    
    raw_reasoning = ' '.join(reasining_lines)
   
    final_resoning = re.sub(r'^\d+\s*[-.:]?\s*', '', raw_reasoning)
    # if len(reasoning_parts) > 1:
            
    #         reasoning = reasoning_parts[1].strip()
    # else:
    #         reasoning = response['answer'].strip()
    # reasoning = response['answer'].split('\n', 1)[1].strip() if '.' in response['answer'] and len(response['answer'].split('.')) > 1 else ""
    final_score = min(score, 10)
    skill_scores.append(final_score)
    print(f"  Score for {skill}: {final_score}/10")
    print(f"  Reasoning: {final_resoning}")
# Overall score calculation
    
overall_score = sum(skill_scores)
n = len(skill_scores)
print(f"Overall Score: {overall_score}/{n * 10}")



  Score for Python: 9/10
  Reasoning: The candidate explicitly mentions Python as a programming language under "TECHNICAL SKILLS," has earned a "Five Star Python Coder" badge from HackerRank, and used Python in multiple projects, providing ample evidence of their proficiency.
  Score for Java: 0/10
  Reasoning: The candidate does not mention any proficiency in Java in the provided context.
  Score for React: 0/10
  Reasoning: React is not mentioned in the provided text.
  Score for PostgreSQL: 0/10
  Reasoning: PostgreSQL is not mentioned at all.
  Score for Cloud Computing (AWS/Azure/GCP): 0/10
  Reasoning: The candidate does not mention any cloud computing platforms (AWS/Azure/GCP) at all.
  Score for Leadership: 0/10
  Reasoning: The candidate does not mention proficiency in Leadership in the provided context.
  Score for Project Management: 0/10
  Reasoning: The candidate does not mention proficiency in project management.
  Score for Machine Learning: 10/10
  Reasoning: Reasoning:

In [ ]:
weaknesses  = []
improvement_suggestions = {}

analysis_result = {
    "missing_skill": ["Mlops", "Devops", 'GenerativeAI']
}

for skill in analysis_result.get('missing_skill', []):
    prompt = f"""
    Analyze why the resume is weak in demonstrating proficiency in "{skill}".

    For your analysis, consider:
    1. What's missing from the resume regarding this skill?
    2. How could it be improved with specific examples?
    3. What specific action items would make this skill stand out?

    Provide your response in this JSON format:
    {{
        "weakness": "A concise description of what's missing or problematic (1-2 sentences)",
        "improvement_suggestions": [
            "Specific suggestion 1",
            "Specific suggestion 2",
            "Specific suggestion 3"
        ],
        "example_addition": "A specific bullet point that could be added to showcase this skill"
    }}

    Return only valid JSON, no other text.
    """
    
    response = ra_chain.invoke({"input": prompt})
    
    # ✅ Extract string answer
    raw_json = response['answer']

    # ✅ Remove backticks and 'json' label
    cleaned_json = re.sub(r'^```json|```$', '', raw_json.strip(), flags=re.MULTILINE).strip()

    try:
        # ✅ Parse string into dictionary
        weakness_data = json.loads(cleaned_json)

        # ✅ Store in desired format
        weakness_detail = {
            "skill": skill,
            "detail": weakness_data.get("weakness", "No specific details provided."),
            "suggestions": weakness_data.get("improvement_suggestions", []),
            "example": weakness_data.get("example_addition", "")
        }

        weaknesses.append(weakness_detail)
        improvement_suggestions[skill] = {
                    "suggestions": weakness_data.get("improvement_suggestions", []),
                    "example": weakness_data.get("example_addition", "")}

    except json.JSONDecodeError as e:
        print(f"JSON parsing failed for skill {skill}: {e}")
        weaknesses.append({
            "skill": skill,
            "detail": raw_json[:200],  # fallback: first 200 characters
            
            "example": ""
        })

resume_weakness = weaknesses

In [65]:
for item in resume_weakness:
    print(f"\n🔧 Skill: {item['skill']}")
    print(f"❌ Weakness: {item['detail']}")
    print(f"📌 Example: {item['example']}")
    print("✅ Suggestions:")
    for s in item['suggestions']:
        print(f"  - {s}")


🔧 Skill: Mlops
❌ Weakness: The resume is weak in demonstrating proficiency in MLOps because it lacks any explicit mention of MLOps practices, tools, or experience with deploying and managing machine learning models in a production environment. While the resume highlights model building and accuracy, it doesn't showcase understanding of the complete ML lifecycle.
📌 Example: Implemented CI/CD pipeline using GitHub Actions and Docker to automate model deployment for the Gemstone Price Predictor, reducing deployment time by 50%.
✅ Suggestions:
  - Include experience with model deployment tools like Docker, Kubernetes, AWS SageMaker, or similar.
  - Mention any involvement in model monitoring, retraining, or version control processes.
  - Specify if any CI/CD pipelines were used for model deployment.

🔧 Skill: Devops
❌ Weakness: The resume is weak in demonstrating proficiency in DevOps because it lacks any explicit mention or demonstration of skills, tools, or experiences related to DevOps

In [78]:
for skill, suggestion_data in improvement_suggestions.items():
    print(f"\n🔧 Skill: {skill}")
    
    print("✅ Suggestions:")
    for suggestion in suggestion_data.get("suggestions", []):
        print(f"  - {suggestion}")
    
    print(f"📌 Example addition: {suggestion_data.get('example', 'No example provided.')}")



🔧 Skill: Mlops
✅ Suggestions:
  - Mention experience with containerization tools like Docker or Kubernetes.
  - Include experience with CI/CD pipelines for model deployment.
  - Add information about model monitoring and performance tracking tools.
📌 Example addition: Implemented CI/CD pipelines using Jenkins to automate model retraining and deployment, reducing deployment time by 50% and ensuring consistent performance monitoring.

🔧 Skill: Devops
✅ Suggestions:
  - Include any experience with cloud platforms (AWS, Azure, GCP) that might involve infrastructure management or deployment.
  - Mention any experience with containerization technologies like Docker or orchestration tools like Kubernetes.
  - If any projects involved automation of deployment or testing, highlight those aspects.
📌 Example addition: Automated deployment of the Gemstone Price Predictor using Docker and GitHub Actions, resulting in a 20% reduction in deployment time.

🔧 Skill: GenerativeAI
✅ Suggestions:
  - Inc